In [13]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('/Users/jvonderhoff/Desktop/FantasyFootball/2024/prod_bestball_2024.db')

query = "SELECT * FROM DK_lineups"
query2 = "SELECT * FROM Underdog_lineups"
dk_df = pd.read_sql(query, conn)
ud_df = pd.read_sql(query2, conn)

In [14]:
unique_lineup_ids = dk_df['lineup_ID'].nunique()
print("Number of unique lineup_ID values:", unique_lineup_ids)


Number of unique lineup_ID values: 28


In [15]:
# Filter lineups where player_position = 'QB'
qb_lineups = dk_df[dk_df['player_position'] == 'QB']

# Count the occurrences of player names where player_position = 'QB'
qb_player_counts = qb_lineups['player_name'].value_counts()
 
print(qb_player_counts)

Drake Maye            8
Trevor Lawrence       5
Jayden Daniels        5
Brock Purdy           5
Jalen Hurts           4
Bryce Young           4
Kyler Murray          4
Deshaun Watson        3
Geno Smith            3
Joe Burrow            3
Aaron Rodgers         3
Matthew Stafford      3
Lamar Jackson         3
Josh Allen            2
Anthony Richardson    2
Justin Herbert        2
Russell Wilson        2
Tua Tagovailoa        1
Gardner Minshew II    1
Daniel Jones          1
Jordan Love           1
J.J. McCarthy         1
Bo Nix                1
Caleb Williams        1
Sam Howell            1
Dak Prescott          1
Derek Carr            1
Jacoby Brissett       1
Kirk Cousins          1
Patrick Mahomes       1
Name: player_name, dtype: int64


In [16]:
qb_lineup_percentages = (qb_player_counts / len(unique_lineup_ids)) * 100
print(qb_lineup_percentages)

TypeError: object of type 'int' has no len()

In [3]:
# QB Stacks

# Group the data by lineup_ID
grouped = dk_df.groupby('lineup_ID')

# Initialize a dictionary to store the relationship between QB and other players on the same team
relationship = {}

# Iterate over each group
for _, group in grouped:
    qb_team = None
    qb_name = None
    
    # Iterate over each row in the group
    for _, row in group.iterrows():
        if row['player_position'] == 'QB':
            # Store the QB's team and name
            qb_team = row['player_team']
            qb_name = row['player_name']
        elif row['player_team'] == qb_team:
            # Check if the player's team matches the QB's team
            if qb_name not in relationship:
                relationship[qb_name] = []
            relationship[qb_name].append(row['player_name'])

# Print the relationship between QB and other players on the same team
for qb_name, players in relationship.items():
    print("Quarterback:", qb_name)
    print("Players on the same team within the lineup:")
    for player in players:
        print(player)
    print()

Quarterback: Jayden Daniels
Players on the same team within the lineup:
Brian Robinson Jr.
Ben Sinnott
Luke McCaffrey
Jahan Dotson
Luke McCaffrey

Quarterback: Aaron Rodgers
Players on the same team within the lineup:
Malachi Corley

Quarterback: Justin Herbert
Players on the same team within the lineup:
Kimani Vidal

Quarterback: Geno Smith
Players on the same team within the lineup:
Noah Fant
Noah Fant

Quarterback: Drake Maye
Players on the same team within the lineup:
Javon Baker
K.J. Osborn
Javon Baker
K.J. Osborn
Austin Hooper

Quarterback: Jalen Hurts
Players on the same team within the lineup:
Dallas Goedert

Quarterback: Kyler Murray
Players on the same team within the lineup:
Trey Benson
Trey Benson

Quarterback: Deshaun Watson
Players on the same team within the lineup:
Elijah Moore
Jamari Thrash

Quarterback: Brock Purdy
Players on the same team within the lineup:
Ricky Pearsall
Jacob Cowing

Quarterback: Daniel Jones
Players on the same team within the lineup:
Daniel Belli

In [4]:
# Create an empty list to store the extracted player information
player_info = []

# Get unique lineup IDs
unique_lineup_ids = dk_df['lineup_ID'].unique()

# Loop through each unique lineup ID
for lineup_id in unique_lineup_ids:
    lineup_players = dk_df[dk_df['lineup_ID'] == lineup_id]
    
    # Count the number of players from each team in the lineup
    team_counts = lineup_players['player_team'].value_counts()
    
    # Filter out the teams with more than one player in the lineup
    teams_with_multiple_players = team_counts[team_counts > 1].index
    
    # Extract player information for teams with more than one player in the lineup
    for team in teams_with_multiple_players:
        team_players = lineup_players[lineup_players['player_team'] == team]
        for index, row in team_players.iterrows():
            player_info.append({
                'Lineup ID': lineup_id,
                'Player Name': row['player_name'],
                'Player Team': row['player_team'],
                'Player Position': row['player_position']
            })

# Create a DataFrame from the extracted player information
player_info_df = pd.DataFrame(player_info)

# Display the DataFrame as a table
player_info_df

,Lineup ID,Player Name,Player Team,Player Position
0,7503db21-be71-4e4d-ba2b-e47bbcff570b,Ja'Lynn Polk,NE,WR
1,7503db21-be71-4e4d-ba2b-e47bbcff570b,Drake Maye,NE,QB
2,7503db21-be71-4e4d-ba2b-e47bbcff570b,Jacoby Brissett,NE,QB
3,7503db21-be71-4e4d-ba2b-e47bbcff570b,Ja'Marr Chase,CIN,WR
4,7503db21-be71-4e4d-ba2b-e47bbcff570b,Tee Higgins,CIN,WR
...,...,...,...,...
306,748782c2-2786-4c89-8874-c2a21d2ca01c,Gardner Minshew II,LV,QB
307,748782c2-2786-4c89-8874-c2a21d2ca01c,Trevor Lawrence,JAX,QB
308,748782c2-2786-4c89-8874-c2a21d2ca01c,Parker Washington,JAX,WR
309,748782c2-2786-4c89-8874-c2a21d2ca01c,Demario Douglas,NE,WR


In [5]:
# Export the DataFrame to a CSV file
player_info_df.to_csv('player_info.csv', index=False)